# Efficient Yelp API Calls

- Darlene Adams

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/dmusl/.secret/yelp_api.json') as f:
    login = json.load(f)

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [4]:
# set our API call parameters and filename before the first call
location = 'Cleveland, OH 44135'
term = 'vegan'

In [5]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_cleveland_vegan.json"
JSON_FILE

'Data/results_in_progress_cleveland_vegan.json'

In [8]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_cleveland_vegan.json already exists.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(term = term, location = location, offset = n_results)

results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
## How many results total?
total_results = results['total']
total_results

664

In [11]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [12]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

34

In [13]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=location,
                                    term=term, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/34 [00:00<?, ?it/s]

In [14]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,VXVrYjLRqNpQY-ogN9ztmQ,cleveland-vegan-lakewood,Cleveland Vegan,https://s3-media4.fl.yelpcdn.com/bphoto/-Do3Co...,False,https://www.yelp.com/biz/cleveland-vegan-lakew...,295,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 41.4855267595181, 'longitude': -8...",[delivery],$$,"{'address1': '17112 Detroit Ave', 'address2': ...",+12162210201,(216) 221-0201,6939.587363
1,3aXpQWD5wGu2TtqnH9rCNA,the-green-kitchen-little-rose-tavern-cleveland,The Green Kitchen @ Little Rose Tavern,https://s3-media4.fl.yelpcdn.com/bphoto/Rpqb_-...,False,https://www.yelp.com/biz/the-green-kitchen-lit...,9,"[{'alias': 'vegan', 'title': 'Vegan'}]",4.0,"{'latitude': 41.45484, 'longitude': -81.79272}",[],NaN,"{'address1': '14206 Lorain Ave', 'address2': N...",+12163311441,(216) 331-1441,4046.613382
2,_6hReDyVSr94W2WMO1LJLw,the-root-cafe-lakewood-2,The Root Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/cOVt1H...,False,https://www.yelp.com/biz/the-root-cafe-lakewoo...,389,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 41.4851374, 'longitude': -81.8020...","[pickup, delivery]",$$,"{'address1': '15118 Detroit Ave', 'address2': ...",+12162264401,(216) 226-4401,7001.012861
3,umYIc-FJg-3tsqoVaM-Mnw,earth-bistro-cafe-cleveland,Earth Bistro Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/wwPaWa...,False,https://www.yelp.com/biz/earth-bistro-cafe-cle...,209,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 41.48653797614105, 'longitude': -...",[],$$,"{'address1': '11100 Clifton Ave', 'address2': ...",+12162019338,(216) 201-9338,8371.716278
4,1zKjojtmAHuVOHhtGVxC2Q,jukebox-cleveland,Jukebox,https://s3-media1.fl.yelpcdn.com/bphoto/5lrHjQ...,False,https://www.yelp.com/biz/jukebox-cleveland?adj...,122,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 41.4895582, 'longitude': -81.7108...","[pickup, delivery]",$$,"{'address1': '1404 W 29th St', 'address2': '',...",+12162067699,(216) 206-7699,11494.343149


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
659,WHlbUb5SNdXDKYfkNDo89A,pickwick-and-frolic-restaurant-and-club-clevel...,Pickwick & Frolic Restaurant and Club,https://s3-media1.fl.yelpcdn.com/bphoto/ldD9Jx...,False,https://www.yelp.com/biz/pickwick-and-frolic-r...,206,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,"{'latitude': 41.4993092, 'longitude': -81.6900...",[delivery],$$,"{'address1': '2035 E 4th St', 'address2': '', ...",+12162417425,(216) 241-7425,13522.281213
660,VDHN8ahFFBX8cC7wSZoUIQ,pierogi-palace-cleveland,Pierogi Palace,https://s3-media2.fl.yelpcdn.com/bphoto/VGyJsm...,False,https://www.yelp.com/biz/pierogi-palace-clevel...,13,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",2.5,"{'latitude': 41.484673, 'longitude': -81.70311}","[pickup, delivery]",$$,"{'address1': '1979 W 25th St', 'address2': 'St...",+12168619800,(216) 861-9800,11672.208471
661,Occo0_pOXAUG-eL6gazeGQ,applebees-grill-bar-cleveland,Applebee's Grill + Bar,https://s3-media3.fl.yelpcdn.com/bphoto/zSB7Lf...,False,https://www.yelp.com/biz/applebees-grill-bar-c...,58,"[{'alias': 'tradamerican', 'title': 'American ...",2.0,"{'latitude': 41.461461, 'longitude': -81.689881}","[pickup, delivery]",$$,"{'address1': '3401 Steelyard Dr', 'address2': ...",+12166613756,(216) 661-3756,11387.395905
662,kBZ-tBkOKGqUOAypk-L9Cg,master-pizza-avon-2,Master Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/AYBAAG...,False,https://www.yelp.com/biz/master-pizza-avon-2?a...,34,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 41.4603089104384, 'longitude': -8...","[pickup, delivery]",$$,"{'address1': '35954 Detroit Rd', 'address2': '...",+14404672424,(440) 467-2424,17119.539810
663,daLUceDcNHEIxLHObOH18g,red-robin-gourmet-burgers-and-brews-independen...,Red Robin Gourmet Burgers and Brews,https://s3-media2.fl.yelpcdn.com/bphoto/SsKHE9...,False,https://www.yelp.com/biz/red-robin-gourmet-bur...,147,"[{'alias': 'burgers', 'title': 'Burgers'}]",2.5,"{'latitude': 41.393489, 'longitude': -81.6429}","[pickup, delivery]",$$,"{'address1': '6750 Rockside Rd', 'address2': '...",+12166420881,(216) 642-0881,14848.497883


In [15]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_in_progress_cleveland_vegan.csv', compression='gzip',index=False)